In [1]:
import tensorflow as tf
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
import cv2
from PIL import Image
import tensorflow as tf
from random import randrange
import pickle
import json
import tensorflow.keras as keras
import os
import sys
import time

In [4]:
def load_shuffled_data():
    """
    Load dataset from pickle file and split features and labels
    returns (X_train, X_test, y_train, y_test)
    """
    start_time = time.time()

    with open("sample_train_img.pkl", 'rb') as f_name:
        train_img = pickle.load(f_name)
        train_img /= 255.0

    with open("sample_train_label.pkl", 'rb') as f_name:
        train_label = pickle.load(f_name)
        train_label = tf.keras.utils.to_categorical(train_label.astype('int'))
        
    
    with open("sample_test_img.pkl", 'rb') as f_name:
        test_img = pickle.load(f_name)
        test_img /= 255.0

    with open("sample_test_label.pkl", 'rb') as f_name:
        test_label = pickle.load(f_name)
        test_label = tf.keras.utils.to_categorical(test_label.astype('int'))

    #Shuffling Images!

    permute_train = np.random.permutation(len(train_label))
    permute_test = np.random.permutation(len(test_label))

    train_img = train_img[permute_train]
    train_label = train_label[permute_train]

    test_img = train_img[permute_train]
    test_label = train_label[permute_train]

    return(train_img, test_img, train_label, test_label)

    

In [5]:
def CNN_model(image_height = 256, image_width = 256, image_channel=1, class_count = 2):
    
    model = Sequential()
    model.add(Conv2D(   filters = 64, 
                        kernel_size=(9,9), 
                        strides = 2, 
                        activation='relu', 
                        kernel_initializer='he_uniform', 
                        input_shape = (image_height, image_width, image_channel)))

    model.add(BatchNormalization())

    #Taking the maximum in that subregion using MaxPooling
    model.add(MaxPool2D(    pool_size = (3, 3),
                            strides = 2))
    #he_uniform: Draw samples from uniform distributions within range of limits created with units

    model.add(Conv2D(   filters = 256, 
                        kernel_size=(5,5), 
                        strides = 1, 
                        activation='relu', 
                        kernel_initializer='he_uniform'))

    model.add(BatchNormalization())
    model.add(MaxPool2D(    pool_size = (3, 3),
                            strides = 2))


    model.add(Conv2D(   filters = 128, 
                        kernel_size=(3,3), 
                        strides = 1, 
                        activation='relu', 
                        kernel_initializer='he_uniform'))

    model.add(BatchNormalization())
    model.add(MaxPool2D(    pool_size = (3, 3),
                            strides = 2))
    
    model.add(Flatten())
    model.add(Dense(    units = 4096, 
                        activation='relu',              
                        kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))   

    model.add(Dense(    units = 1024, 
                        activation='relu', 
                        kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))   

    model.add(Dense(    units = 256, 
                            activation='relu', 
                            kernel_initializer='he_uniform'))

    model.add(Dropout(0.5)) 
    model.add(Dense(    units = 2, 
                        activation='softmax'))
 

    #Compiling:
    opt = SGD(lr = 0.001, momentum = 0.9)
    model.compile(  optimizer=opt, 
                    loss='categorical_crossentropy',
                    metrics = ['accuracy'])

    return model

In [11]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 124, 64)      5248      
_________________________________________________________________
batch_normalization (BatchNo (None, 124, 124, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 61, 61, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 57, 57, 256)       409856    
_________________________________________________________________
batch_normalization_1 (Batch (None, 57, 57, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 128)       2

In [6]:
cnn_model = CNN_model()
train_x, test_x, train_y, test_y = load_shuffled_data()

In [7]:
#Fit Model!
#Replace text_x, test_y with validation data collected!
es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=10)
mc = ModelCheckpoint('CNN_v2_Trial_2.h5', monitor='val_accuracy', mode='max', verbose=2, save_best_only=True)
history = cnn_model.fit(train_x, train_y, epochs = 10, batch_size = 32, validation_data = (test_x, test_y), verbose = 2, callbacks = [es, mc])

Epoch 1/10

Epoch 00001: val_accuracy improved from -inf to 0.56667, saving model to CNN_v2_Trial_2.h5
6/6 - 24s - loss: 3.3685 - accuracy: 0.5444 - val_loss: 0.6996 - val_accuracy: 0.5667
Epoch 2/10

Epoch 00002: val_accuracy did not improve from 0.56667
6/6 - 26s - loss: 3.2210 - accuracy: 0.5889 - val_loss: 1.3761 - val_accuracy: 0.5056
Epoch 3/10

Epoch 00003: val_accuracy did not improve from 0.56667
6/6 - 22s - loss: 4.4941 - accuracy: 0.6333 - val_loss: 1.3236 - val_accuracy: 0.5167
Epoch 4/10

Epoch 00004: val_accuracy improved from 0.56667 to 0.71667, saving model to CNN_v2_Trial_2.h5
6/6 - 24s - loss: 3.0907 - accuracy: 0.6833 - val_loss: 0.5716 - val_accuracy: 0.7167
Epoch 5/10

Epoch 00005: val_accuracy did not improve from 0.71667
6/6 - 20s - loss: 3.1725 - accuracy: 0.7167 - val_loss: 2.5869 - val_accuracy: 0.5000
Epoch 6/10

Epoch 00006: val_accuracy did not improve from 0.71667
6/6 - 24s - loss: 3.5473 - accuracy: 0.6667 - val_loss: 0.9731 - val_accuracy: 0.5389
Epoch 7

In [8]:
#Evaluate Data
loss, acc = cnn_model.evaluate(test_x, test_y, verbose = 0)

In [9]:
print(acc)

0.605555534362793
